In [3]:
import json
import random

In [2]:
train_path = "./data/data.json"

with open(train_path, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))

random.shuffle(data)
print('Dataset is shuffled...')

train_dataset = data[:int(len(data)*0.9)]
test_dataset = data[int(len(data)*0.9):]

print('Dataset is splitted...')

Dataset size: 57
Dataset is shuffled...
Dataset is splitted...


In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

c:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\gliner_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
CUDA version: 12.1
GPU count: 1
GPU name: NVIDIA RTX 4500 Ada Generation


In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model = GLiNER.from_pretrained("almanach/camembert-bio-gliner-v0.1")

cuda:0


Fetching 4 files: 100%|██████████| 4/4 [00:00<?, ?it/s]
c:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\gliner_env\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\gliner_env\Lib\site-packages\gliner\model.py:483: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling.

In [6]:
# use it for better performance, it mimics original implementation but it's less memory efficient
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [7]:
# Optional: compile model for faster training
model.to(device)
print("done")

done


In [8]:
data_collator

In [12]:
# calculate number of epochs
num_steps = 500
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

c:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\gliner_env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

 86%|████████▌ | 500/581 [01:57<00:18,  4.48it/s]

{'loss': 6.1245, 'grad_norm': 86.10486602783203, 'learning_rate': 1.5517241379310346e-06, 'epoch': 71.43}


                                                 
 86%|████████▌ | 500/581 [01:57<00:18,  4.48it/s]

{'eval_loss': 73.24057006835938, 'eval_runtime': 0.0436, 'eval_samples_per_second': 137.703, 'eval_steps_per_second': 22.95, 'epoch': 71.43}


100%|██████████| 581/581 [02:16<00:00,  4.25it/s]

{'train_runtime': 136.8578, 'train_samples_per_second': 30.93, 'train_steps_per_second': 4.245, 'train_loss': 5.602493680077453, 'epoch': 83.0}


TrainOutput(global_step=581, training_loss=5.602493680077453, metrics={'train_runtime': 136.8578, 'train_samples_per_second': 30.93, 'train_steps_per_second': 4.245, 'total_flos': 0.0, 'train_loss': 5.602493680077453, 'epoch': 83.0})

In [6]:
trained_model = GLiNER.from_pretrained("models/BTB_gliner/checkpoint-1300", load_tokenizer=True)

config.json not found in C:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\src\finetuning\models\BTB_gliner\checkpoint-1300
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\gliner_env\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\benysar\Documents\GitHub\GLiner-TransbronchialBiopsy\gliner_env\Lib\site-packages\gliner\model.py:483: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SE

In [7]:
text = """
1/ Lavage broncho alvéolaire :
Liquide hypercellulaire avec légère polynucléose à polynucléaires neutrophiles sans agent pathogène
retrouvé.
2/ Biopsies transbronchiques : 7 fragments.
Absence de rejet aigu cellulaire bronchiolaire ou parenchymateux. A0 B0
Absence de lésions évocatrices de rejet aigu humoral.
Absence de lésions évocatrices de rejet chronique.
Absence d'inclusion virale et notamment d’inclusion de type CMV."""

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet Chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]# for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

7 => Nombre Total De Fragments
B0 => Grade B
lésions évocatrices de rejet aigu humoral => Éosinophilie
lésions évocatrices de rejet chronique => Éosinophilie


In [50]:
text = """
LBA : Liquide broncho-alvéolaire de cellularité élevée avec importante polyunucléose à
polynucléaires neutrophiles parfois altérés.
Présence d’un amas bactérien extracellulaire.

Biopsie transbronchique (LID) :
Fragments biopsiques de bonne taille ayant intéressé des berges bronchiolaires ou de
petites bronches dépourvues de tout argument morphologique pour un éventuel rejet
aigu bronchiolaire.
A noter l’existence d’un minime infiltrat inflammatoire polymorphe au sein du chorion
témoignant de discrètes lésions de bronchiolite aiguë sans caractère de spécificité.
Le parenchyme pulmonaire intéressé par ces prélèvements ne montre pas d’argument
morphologique en faveur d’un éventuel rejet aigu parenchymateux minime.
Absence d’inclusion virale et notamment d’inclusion à CMV.
A0 B0"""

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet Chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

LID => Site
A0 => Grade A
B0 => Grade B


In [8]:
text = """
Biopsie transbronchique (LID) :
Fragments biopsiques de bonne taille ayant intéressé des berges bronchiolaires ou de
petites bronches dépourvues de tout argument morphologique pour un éventuel rejet
aigu bronchiolaire.
A noter l’existence d’un minime infiltrat inflammatoire polymorphe au sein du chorion
témoignant de discrètes lésions de bronchiolite aiguë sans caractère de spécificité.
Le parenchyme pulmonaire intéressé par ces prélèvements ne montre pas d’argument
morphologique en faveur d’un éventuel rejet aigu parenchymateux minime.
Absence d’inclusion virale et notamment d’inclusion à CMV.
A0 B0. """

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet Chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

LID => Site
B0 => Grade B


In [9]:
text = """
1/ Biopsie transbronchique (LM) :
1 fragment biopsique ayant intéressé la plèvre viscérale, sans parenchyme
pulmonaire, ne permettant pas une étude histologique contributive.

2/ Lavage broncho alvéolaire :
Lavage de richesse cellulaire légèrement augmentée, sans anomalie de la formule
cytologique.
Absence d’agent pathogène opportuniste. """

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",

]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

LM => Site
1 => Nombre Total De Fragments


In [10]:
text = """
 BIOPSIE TRANSBRONCHIQUE

Renseignements cliniques : re-transplantation pulmonaire. Bilan à J38
Quatre fragments de grande taille avec des berges bronchiolaires sans anomalie. Dans le chorion,
quelques dépôts d’anthracose. Infiltrat lymphocytaire focal à l’interface avec le parenchyme pulmonaire.
Ce dernier montre des alvéoles normales, des cloisons discrètement congestives ou parfois avec discret
épaississement fibreux. Rares macrophages intra alvéolaires. Les nombreuses sections vasculaires visibles
dans ce parenchyme sont dépourvues d’infiltrat inflammatoire mononucléé au pourtour. Absence
d’inclusion de type viral.  """

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet Chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Quatre => Nombre Total De Fragments
des cloisons discrètement congestives => Lésion Septale


In [11]:
text = """
I - Biopsie transbronchique (lobe inférieur droit) :
1 fragment biopsique ayant intéressé un parenchyme pulmonaire sans signe de rejet
aigu cellulaire parenchymateux ou bronchiolaire (grade A0 B0).
Absence d’inclusion virale de type CMV sur le matériel transmis à examiner. """

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

lobe inférieur droit => Site
1 => Nombre Total De Fragments
A0 => Grade A
B0 => Grade B


In [12]:
text = """
Biopsies transbronchiques : présence sur un seul plan de coupes des 6 fragments, d’un
infiltrat mononucléé entourant une structure vasculaire intra parenchymateuse avec
réaction d’endothélite pouvant témoigner d’un rejet aigu cellulaire minime. Absence de
signe de rejet bronchiolaire. Absence d’inclusion virale, notamment de type CMV.
A1 focal B0. """

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Grade A",
    "Grade B",
    "Rejet Chronique",

]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

6 => Nombre Total De Fragments
A1 => Grade A
B0 => Grade B


In [13]:
text = """
I – Biopsies bronchiques étagées droites + biopsie transbronchique :
Parenchyme pulmonaire d’architecture normale.
Ulcération bronchique, avec importants remaniements inflammatoires non spécifiques
du chorion.
Absence de nécrose en carte de géographie, absence de lésion de vascularite ou de micro
abcès à polynucléaires sur le matériel transmis à examiner."""

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Ulcération bronchique => Lésion Intra-Alvéolaire
lésion de vascularite => Lésion Intra-Alvéolaire


In [14]:
text = """
1/ Biopsies transbronchiques (LID – LM) :
9 fragments biopsiques ayant intéressé un parenchyme pulmonaire sans argument
morphologique en faveur d’un rejet aigu cellulaire parenchymateux ou bronchiolaire (A0 et B0).
Aspect d’alvéolite macrophagique, associée à une discrète hyperplasie pneumocytaire et à
quelques fibromes végétants endo alvéolaires. 
Absence d’inclusion virale de type CMV."""

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

LID => Site
LM => Site
9 => Nombre Total De Fragments
B0 => Grade B
hyperplasie pneumocytaire => Lésion Intra-Alvéolaire
quelques fibromes végétants endo alvéolaires => Pneumonie Organisée


In [15]:
text = """
1. Biopsies transbronchiques (LID – LM) : 6 fragments biopsiques ayant intéressé un parenchyme pulmonaire présentant : - 2 manchons lymphocytaires circonférenciels péri vasculaires évoquant un rejet aigu parenchymateux de grade A1+. Berges bronchiolaires sans anomalie morphologique notable (B0). - Des aspects d’alvéolite fibrineuse, macrophagique et végétante, associées à une hyperplasie pneumocytaire, pouvant être la traduction d’une agression du compartiment endo alvéolaire secondaire à un œdème de reperfusion. Absence d’inclusion virale de type CMV."""

# Labels for entity prediction
labels = [
    "Site",
    "Nombre Total De Fragments",
    "Nombre Total De Fragments Alvéolés",
    "Grade A",
    "Grade B",
    "Rejet chronique",
    "Coloration C4d",
    "Lésion Septale",
    "Lésion Intra-Alvéolaire",
    "Éosinophilie",
    "Pneumonie Organisée",
    "DAD",
    "Infection",
    "Autre Pathologie"
]

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.3)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

LID => Site
LM => Site
6 => Nombre Total De Fragments
A1+ => Grade A
B0 => Grade B
Des aspects d’alvéolite fibrineuse, macrophagique et végétante => Lésion Intra-Alvéolaire
hyperplasie pneumocytaire => Lésion Intra-Alvéolaire
agression du compartiment endo alvéolaire => Lésion Intra-Alvéolaire
œdème de reperfusion => Lésion Intra-Alvéolaire


In [83]:
import pandas as pd

def predict_entities_for_texts(texts, model, labels, threshold=0.5, output_filename="entities_predictions.xlsx"):
    # Prepare a list to store results
    all_results = []

    # Loop over each text and predict entities
    for text in texts:
        entities = model.predict_entities(text, labels, threshold)
        
        # Create a dictionary for each entity with 'text' and 'label' columns
        result = {'Text': text}
        for label in labels:
            result[label] = ""
        
        # Add the predicted entities and their corresponding labels
        for entity in entities:
            result[entity["label"]] = entity["text"]
        
        all_results.append(result)
    
    # Create a DataFrame
    df = pd.DataFrame(all_results)
    
    # Save the DataFrame to an Excel file
    df.to_excel(output_filename, index=False, engine="openpyxl")
    print(f"Predictions saved to {output_filename}")

# Example usage:
texts = [
    """
    1/ Lavage broncho alvéolaire :
    Liquide hypercellulaire avec légère polynucléose à polynucléaires neutrophiles sans agent pathogène
    retrouvé.
    2/ Biopsies transbronchiques : 7 fragments.
    Absence de rejet aigu cellulaire bronchiolaire ou parenchymateux. A0 B0
    Absence de lésions évocatrices de rejet aigu humoral.
    Absence de lésions évocatrices de rejet chronique.
    Absence d'inclusion virale et notamment d’inclusion de type CMV.
    """,

    """
    LBA : Liquide broncho-alvéolaire de cellularité élevée avec importante polyunucléose à
    polynucléaires neutrophiles parfois altérés.
    Présence d’un amas bactérien extracellulaire.

    Biopsie transbronchique (LID) :
    Fragments biopsiques de bonne taille ayant intéressé des berges bronchiolaires ou de
    petites bronches dépourvues de tout argument morphologique pour un éventuel rejet
    aigu bronchiolaire.
    A noter l’existence d’un minime infiltrat inflammatoire polymorphe au sein du chorion
    témoignant de discrètes lésions de bronchiolite aiguë sans caractère de spécificité.
    Le parenchyme pulmonaire intéressé par ces prélèvements ne montre pas d’argument
    morphologique en faveur d’un éventuel rejet aigu parenchymateux minime.
    Absence d’inclusion virale et notamment d’inclusion à CMV.
    A0 B0
    """,

    """
    Biopsie transbronchique (LID) :
    Fragments biopsiques de bonne taille ayant intéressé des berges bronchiolaires ou de
    petites bronches dépourvues de tout argument morphologique pour un éventuel rejet
    aigu bronchiolaire.
    A noter l’existence d’un minime infiltrat inflammatoire polymorphe au sein du chorion
    témoignant de discrètes lésions de bronchiolite aiguë sans caractère de spécificité.
    Le parenchyme pulmonaire intéressé par ces prélèvements ne montre pas d’argument
    morphologique en faveur d’un éventuel rejet aigu parenchymateux minime.
    Absence d’inclusion virale et notamment d’inclusion à CMV.
    A0 B0.
    """,

    """
    1/ Biopsie transbronchique (LM) :
    1 fragment biopsique ayant intéressé la plèvre viscérale, sans parenchyme
    pulmonaire, ne permettant pas une étude histologique contributive.

    2/ Lavage broncho alvéolaire :
    Lavage de richesse cellulaire légèrement augmentée, sans anomalie de la formule
    cytologique.
    Absence d’agent pathogène opportuniste.
    """,

    """
    BIOPSIE TRANSBRONCHIQUE

    Renseignements cliniques : re-transplantation pulmonaire. Bilan à J38
    Quatre fragments de grande taille avec des berges bronchiolaires sans anomalie. Dans le chorion,
    quelques dépôts d’anthracose. Infiltrat lymphocytaire focal à l’interface avec le parenchyme pulmonaire.
    Ce dernier montre des alvéoles normales, des cloisons discrètement congestives ou parfois avec discret
    épaississement fibreux. Rares macrophages intra alvéolaires. Les nombreuses sections vasculaires visibles
    dans ce parenchyme sont dépourvues d’infiltrat inflammatoire mononucléé au pourtour. Absence
    d’inclusion de type viral.
    """,

    """
    I - Biopsie transbronchique (lobe inférieur droit) :
    1 fragment biopsique ayant intéressé un parenchyme pulmonaire sans signe de rejet
    aigu cellulaire parenchymateux ou bronchiolaire (grade A0 B0).
    Absence d’inclusion virale de type CMV sur le matériel transmis à examiner.
    """,

    """
    Biopsies transbronchiques : présence sur un seul plan de coupes des 6 fragments, d’un
    infiltrat mononucléé entourant une structure vasculaire intra parenchymateuse avec
    réaction d’endothélite pouvant témoigner d’un rejet aigu cellulaire minime. Absence de
    signe de rejet bronchiolaire. Absence d’inclusion virale, notamment de type CMV.
    A1 focal B0.
    """,

    """
    I – Biopsies bronchiques étagées droites + biopsie transbronchique :
    Parenchyme pulmonaire d’architecture normale.
    Ulcération bronchique, avec importants remaniements inflammatoires non spécifiques
    du chorion.
    Absence de nécrose en carte de géographie, absence de lésion de vascularite ou de micro
    abcès à polynucléaires sur le matériel transmis à examiner.
    """,

    """
    1/ Biopsies transbronchiques (LID – LM) :
    9 fragments biopsiques ayant intéressé un parenchyme pulmonaire sans argument
    morphologique en faveur d’un rejet aigu cellulaire parenchymateux ou bronchiolaire (A0 et B0).
    Aspect d’alvéolite macrophagique, associée à une discrète hyperplasie pneumocytaire et à
    quelques fibromes végétants endo alvéolaires. 
    Absence d’inclusion virale de type CMV.
    """,

    """
    Lavage broncho-alvéolaire : prélèvement peu cellulaire avec abondant mucus, cellules
    cylindrique et hyperleucocytose modérée (aspiration ?). Absence d’agent pathogène
    spécifique mis en évidence sur les colorations cytochimiques.

    Biopsies transbronchiques : signe histologique en faveur d’un rejet aiguë cellulaire
    parenchymateux léger. Absence de rejet bronchiolaire. Absence d’inclusion virale,
    notamment de type CMV.
    A1 B0
    """
]


labels = [
    "Site", "Nombre Total De Fragments", "Nombre Total De Fragments Alvéolés", "Grade A", "Grade B", "Rejet Chronique",
    "Coloration C4d", "Lésion Septale", "Lésion Intra-Alvéolaire", "Éosinophilie", "Pneumonie Organisée", "DAD", "Infection", "Autre Pathologie"
]

# Assuming 'trained_model' is your trained entity prediction model
predict_entities_for_texts(texts, trained_model, labels)


Predictions saved to entities_predictions.xlsx
